# Ejemplo de Red Convolucional con TensorFlow

In [1]:
%reset
%matplotlib inline

# importo tensorflow
import tensorflow as tf
# Siempre util para testear... no se si lo uso acá
import numpy as np
# Herramientas para interactuar con el sistema operativo (opcionales para acomodar cosas)
import os
import shutil



Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Importamos el dataset MINST

In [2]:
# De < http://yann.lecun.com/exdb/mnist/ > descargar los 4 archivos:
#
# train-images-idx3-ubyte.gz:  training set images (9912422 bytes)
# train-labels-idx1-ubyte.gz:  training set labels (28881 bytes)
# t10k-images-idx3-ubyte.gz:   test set images (1648877 bytes)
# t10k-labels-idx1-ubyte.gz:   test set labels (4542 bytes) 

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Definición de Parámetros

In [3]:
# Cantidad de píxeles en cada imagen del set
Tam_imagen = 784
# Tamaño en ambas direcciones de la imagen
Tam_imagen_2D = [28,28]
# Cantidad de canales, es blanco y negros => uno solo
canales_imagen = 1
# Cantidad de clases objetivo, 10 (numeros del 0 al 9)
Cant_salidas = 10

# Tamaño del kernel que vamos a utilizar (uno solo para todos los layers)
kernel_size = [5, 5]
# Cantidad de canales por layer, es decir, filtros en paralelo
Layer_sizes = [16, 32]

# En la salida va a haber un layer totalemente conectado, definimos su numero de neuronas
hidden_fc_neurons = 1024

# Cantidad de pasos a entrenar
pasos_entrenamiento = 20000
# Tamaño del batch a utilizar
batch_size = 50

### Algunas funciones útiles

In [4]:
# Crear variable de peso con pesos iniciales normales y desvio 0.1
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)
# Crear variable de bias con peso inicial en 0.1
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


# Función que calcula mucha info de las variables para TensoBoard
def variable_summaries(var, nombre='summaries'):
  #"""Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope(nombre):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)


# Genero el modelo

Primero declaramos los "placeholders". Estas variables actuan como los canales de entrada a nuestro grafo. Cuando su dimensión es "None" quiere decir que su tamaño se define al momento que cargamos un valor allí. Usaremos ese lugar para poner un batch size que puede ser variable.

In [5]:
# Entrada de imágenes
x = tf.placeholder(tf.float32, shape=[None, Tam_imagen])
# Clases objetivo
y = tf.placeholder(tf.float32, shape=[None, Cant_salidas])
#fase = tf.placeholder(tf.bool)

Luego generamos todas las varibales a "aprender". Todas las variables son entrenables por defecto.

In [6]:
# Peso del primer layer convolucional, es del tamaño del kernel, por la cantidad 
# de canales por la cantidad de filtros en paralelo
W_conv0 = weight_variable([kernel_size[0], kernel_size[1], canales_imagen, Layer_sizes[0]])
variable_summaries(W_conv0, "Pesos_conv_0")
# Bias, uno por tipo de filtro aplicado
b_conv0 = bias_variable([Layer_sizes[0]])
variable_summaries(b_conv0, "Bias_conv_0")

# idem pero ahora tengo tantos canales de entrada como de salida en el último nivel
W_conv1 = weight_variable([kernel_size[0], kernel_size[1], Layer_sizes[0], Layer_sizes[1]])
variable_summaries(W_conv1, "Pesos_conv_1")
b_conv1 = bias_variable([Layer_sizes[1]])
variable_summaries(b_conv1, "Bias_conv_1")

# Calculo la cantidad de caracteristicas que voy a tener al final.
# Esta dado por lo que se redijo la imagen y la cantidad de canales finales
tma_imag_fin = [Tam_imagen_2D[0]/(2**len(Layer_sizes)) , Tam_imagen_2D[1]/(2**len(Layer_sizes))]
cant_caract_conv = int(tma_imag_fin[0] * tma_imag_fin[1] * Layer_sizes[-1])

# Genero los pesos de la red totalmente conectada, como siempre
W_fc = weight_variable([cant_caract_conv, hidden_fc_neurons])
variable_summaries(W_fc, "Pesos_Fully_con_oculto")
b_fc = bias_variable([hidden_fc_neurons])
variable_summaries(b_fc, "Bias_Fully_con_oculto")

# Pesos del layer de salida que tiene tantas neuronas como clases a calsificar
W_out = weight_variable([hidden_fc_neurons, Cant_salidas])
variable_summaries(W_out, "Pesos_Fully_con_salida")
b_out = bias_variable([Cant_salidas])
variable_summaries(b_out, "Bias_Fully_con_salida")

Luego armamo la interacción de todo el grafo. Debemos definir como llegamos desde nuestro placeholder de entrada hasta la salida y como comparar esta con el placeholder de clases objetivo

In [7]:
with tf.name_scope('Primer_Layer_Convolucional'):
    # Paso la imagen de formato vector a imagen (de 1D a 2D)
    x_image = tf.reshape(x, [-1, Tam_imagen_2D[0], Tam_imagen_2D[1], 1])

    # Aplico convolución
    s_conv0 = tf.nn.conv2d(x_image, W_conv0, strides=[1, 1, 1, 1], padding='SAME')
    # Agrego la normalización por batch (para regularizar)
    #s_conv0 = tf.contrib.layers.batch_norm(s_conv0,center=True,scale=True,is_training=fase)
    # Agrego peso y alinealidad
    h_conv0 = tf.nn.relu( s_conv0 + b_conv0)
    # Aplico el pooling
    h_pool0 = tf.nn.max_pool(h_conv0, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

with tf.name_scope('Segundo_Layer_Convolucional'):
    # Idem al anterior
    s_conv1 = tf.nn.conv2d(h_pool0, W_conv1, strides=[1, 1, 1, 1], padding='SAME')
    #s_conv1 = tf.contrib.layers.batch_norm(s_conv1,center=True,scale=True,is_training=fase)
    h_conv1 = tf.nn.relu( s_conv1 + b_conv1)
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


with tf.name_scope('Layer_Denso'):
    # El layer totalmente conectado necesita ver nuevamente la imagen como vctor
    h_pool1_flat = tf.reshape(h_pool1, [-1, cant_caract_conv])
    # Aplico la multiplicación y alinealidad
    h_0 = tf.nn.relu(tf.matmul(h_pool1_flat, W_fc) + b_fc)
    
with tf.name_scope('Layer_Salida'):
    # Este pequeño layer funciona como traducción final del numero de features 
    # obtenidas en el último layer a una salida por clase objetivo.
    y_conv = tf.matmul(h_0, W_out) + b_out

with tf.name_scope('Metrica_Error'):
    # Usamos la entropia cruzada para calcular nuestro error
    cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))
    
    # Sacamos una métrica más legible humanamente, el porcentaje de error
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    # Lo agregamos al sumario
    tf.summary.scalar('accurracy', accuracy)


with tf.name_scope('Entrenador'):
    # Agregamos un entrenador tipo ADAM, que minimize la entropia cruzada.
    # Solo se va a encargar de derivar la expresión "cross_entropy" en función
    # de todas las variables que intervengan
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
#extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



## Inicializo la sesión de TensorFlow

In [8]:
# Declaramos la operación para guardar todo (en caso de querer conservar el modelo entrenado)
saver = tf.train.Saver()
# Operación de inicialización de variables (si o si va)
init = tf.global_variables_initializer()
# Activamos la sesión
session = tf.InteractiveSession()
# Inicializamos el grafo
session.run(init)
# Declaramos la salida del tensorboard y construimos la visualización
merged = tf.summary.merge_all()
train_writer_path = 'Tensorboard_out' + '/test'
if os.path.exists(train_writer_path):
    shutil.rmtree(train_writer_path)
    print("Eliminado sumario de entrenamiento anterior.")
train_writer = tf.summary.FileWriter(train_writer_path,session.graph)

Eliminado sumario de entrenamiento anterior.


# ¡ENTRENAMOS!

In [10]:
for i in range(pasos_entrenamiento):
    # Saco un minibatch
    batch = mnist.train.next_batch(batch_size)
    
    if i % 100 == 0:
        
        # Seteo las entradas del grafo
        #feed_dict={x: batch[0], y: batch[1], fase : True}
        feed_dict={x: batch[0], y: batch[1]}
        
        # Ejecuto los nodos "accuracy" y "merged" para calcular la presición actual y cargar los sumarios en tensorboard
        #train_accuracy, _, sumario = session.run([accuracy, extra_update_ops, merged], feed_dict)
        train_accuracy, sumario = session.run([accuracy, merged], feed_dict)
        
        # Agrego lo que obtuve al tensorboard
        train_writer.add_summary(sumario, i)
        
        print('step %d, training accuracy %g' % (i, train_accuracy))
        
    # Entreno ejecutando el paso de entrenamiento, que va a calcular el gradiente, el paso y aplicarlo.
    session.run([train_step], feed_dict)

step 0, training accuracy 0
step 100, training accuracy 0.74
step 200, training accuracy 0.76
step 300, training accuracy 0.82
step 400, training accuracy 0.76
step 500, training accuracy 0.84
step 600, training accuracy 0.86
step 700, training accuracy 0.82
step 800, training accuracy 0.84
step 900, training accuracy 0.82
step 1000, training accuracy 0.84
step 1100, training accuracy 0.88
step 1200, training accuracy 0.8
step 1300, training accuracy 0.88
step 1400, training accuracy 0.9
step 1500, training accuracy 0.86
step 1600, training accuracy 0.86
step 1700, training accuracy 0.9
step 1800, training accuracy 0.84
step 1900, training accuracy 0.88
step 2000, training accuracy 0.9
step 2100, training accuracy 0.84
step 2200, training accuracy 0.92
step 2300, training accuracy 0.86
step 2400, training accuracy 0.94
step 2500, training accuracy 0.84
step 2600, training accuracy 0.9
step 2700, training accuracy 0.86
step 2800, training accuracy 0.88
step 2900, training accuracy 0.9
s

# Calculamos el error de validación

In [11]:
#feed_dict={ x: mnist.test.images, y: mnist.test.labels , fase : False}
feed_dict={ x: mnist.test.images, y: mnist.test.labels}
accuracy_Val = session.run([accuracy],feed_dict)
print('Validation accuracy %s' % accuracy_Val)

Validation accuracy [0.9371]
